In [2]:
import json, os,re
from typing import List
from tqdm import tqdm
from rapidfuzz import process
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os
import typing
from tqdm import tqdm



In [44]:
# Create a map of phenopackets to disease id map
# iterate the phenopacket-store folder get all json files
import glob
p2d = {}
patient_filename = {}
phenopacket_store = "../phenopacket-store"
# Recursively find all json files in subfolders
phenopacket_files = [
    os.path.relpath(f, phenopacket_store)
    for f in glob.glob(os.path.join(phenopacket_store, "**", "*.json"), recursive=True)
]
for file in phenopacket_files:
    with open(os.path.join(phenopacket_store, file)) as f:
        data = json.load(f)
        patient_id = data.get("id", "")
        patient_filename[patient_id] = os.path.join("phenopacket-store", file)
        diseases = data.get("diseases", [])
        disease_ids = [d.get("term", {}).get("id", "") for d in diseases if "term" in d]
        if disease_ids:
            print(disease_ids) if len(disease_ids) > 1 else None
            p2d[patient_id] = disease_ids

len(p2d)


6033

In [4]:
list(p2d.values())[0:5]

[['OMIM:620371'],
 ['OMIM:620371'],
 ['OMIM:620371'],
 ['OMIM:620371'],
 ['OMIM:620371']]

In [5]:
# Install oaklib if not already installed
# !pip install oaklib

from oaklib import get_adapter

# Path to your downloaded ORDO.owl file
ordo_owl_path = "../data/ordo_orphanet.owl"

# Load the ontology using OAK
adapter = get_adapter(f"sqlite:obo:ordo")

# Get all entities and print the size
entities = list(adapter.entities())
print(f"Number of ORDO entities: {len(entities)}")

# Iterate every term in ORDO and collect OMIM to Orphanet mappings using only oio:hasDbXref predicate
omim_to_orpha = {}
problems = {}


for ordo_term in tqdm(entities, desc="ORDO terms"):
    # For each term, get all SSSOM mappings with predicate 'oio:hasDbXref' (case-insensitive)
    if "ordo" in ordo_term.lower():
        for m in adapter.sssom_mappings(ordo_term):
            if m.predicate_id and m.predicate_id.lower() == 'oio:hasdbxref' or m.predicate_id.lower() == "skos:exactmatch":
                # OMIM mappings are usually in the object_id or subject_id
                if "omim" in m.object_id.lower():
                    omim_id = m.object_id
                    orpha_id = m.subject_id.replace("ORDO:", "ORPHA:") if m.subject_id.startswith("ORDO:") or m.subject_id.startswith("ORPHA:") else m.subject_id
                    if omim_id in omim_to_orpha:
                        # If already mapped, flag as a problem
                        problems[omim_id] = problems.get(omim_id, []) + [orpha_id]
                    else:
                        omim_to_orpha[omim_id] = orpha_id

/Users/gargam/Develop/pheval.llm/newerp3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of ORDO entities: 18503


ORDO terms: 100%|██████████| 18503/18503 [06:49<00:00, 45.18it/s]


In [10]:
print(omim_to_orpha.items())

dict_items([('OMIM:300650', 'ORPHA:1000'), ('OMIM:105150', 'ORPHA:100008'), ('OMIM:130900', 'ORPHA:100032'), ('OMIM:104510', 'ORPHA:100034'), ('OMIM:606483', 'ORPHA:100043'), ('OMIM:606482', 'ORPHA:100044'), ('OMIM:608323', 'ORPHA:100045'), ('OMIM:607791', 'ORPHA:100046'), ('OMIM:106100', 'ORPHA:100050'), ('OMIM:610618', 'ORPHA:100054'), ('OMIM:600430', 'ORPHA:1001'), ('OMIM:181250', 'ORPHA:1003'), ('OMIM:203550', 'ORPHA:1005'), ('OMIM:104130', 'ORPHA:1008'), ('OMIM:612740', 'ORPHA:100924'), ('OMIM:309548', 'ORPHA:100973'), ('OMIM:182600', 'ORPHA:100984'), ('OMIM:182601', 'ORPHA:100985'), ('OMIM:270800', 'ORPHA:100986'), ('OMIM:600363', 'ORPHA:100988'), ('OMIM:603563', 'ORPHA:100989'), ('OMIM:604187', 'ORPHA:100991'), ('OMIM:604805', 'ORPHA:100993'), ('OMIM:605280', 'ORPHA:100994'), ('OMIM:605229', 'ORPHA:100995'), ('OMIM:270700', 'ORPHA:100996'), ('OMIM:300266', 'ORPHA:100997'), ('OMIM:270685', 'ORPHA:100998'), ('OMIM:607152', 'ORPHA:100999'), ('OMIM:125370', 'ORPHA:101'), ('OMIM:1041

In [13]:
# Now map patient_id to Orphanet using the OMIMs in p2d
patient_to_orpha = {}
problems = {}
for patient_id, omim_ids in tqdm(p2d.items()):
    mapped = []
    for omim in omim_ids:
        if omim in omim_to_orpha:
            mapped.append(omim_to_orpha.get(omim))
    if len(mapped) == 1:
        patient_to_orpha[patient_id] = mapped[0]
    elif len(mapped) > 1:
        print(mapped)
        problems[patient_id] = mapped
        print(f"Problem: Patient {patient_id} OMIMs {omim_ids} map to multiple Orphanet IDs: {mapped}")
if problems:
    print(f"\nProblems found: {problems}")

100%|██████████| 6033/6033 [00:00<00:00, 1489974.45it/s]


In [17]:
print(len(patient_to_orpha))

3185


In [ ]:
phenopacket_out = "../analysis_out/phenopacket_subsets/prevalence/"
                                   
def normalize(s):
    return re.sub(r'[^a-zA-Z0-9]', '', s).lower()

prevalence_map: typing.Dict[str, float] = {
    "<1 / 1 000 000": 1e-6,
    "1-9 / 1 000 000": 5e-6,
    "1-9 / 100 000": 5e-5,
    '1-5 / 10 000':2.5e-4,
    '6-9 / 10 000':7.5e-4,
    '>1 / 1000':1e-3,
    'Not yet documented': 1e-6,
    "Unknown": 1e-6,
}

prevalence_category: typing.Dict[str, str] = {
    "<1 / 1 000 000": "RARE",
    "1-9 / 1 000 000": "RARE",
    "1-9 / 100 000": "RARE",
    '1-5 / 10 000': "COMMON",
    '6-9 / 10 000':"COMMON",
    '>1 / 1000': "COMMON",
}

class OrphaPrev:
    def __init__(self) -> None:
        self.download_orpha_file("en_product9_prev.xml") # prevalence
        self._prevalence_df = self.xml_to_pd_prevalence("../data/en_product9_prev.xml")
        self.download_orpha_file("en_product9_ages.xml") # onset
        self._onset_df = self.xml_to_pd_onset("../data/en_product9_ages.xml")
        self.download_orpha_file("en_product6.xml") # genes
        self._genes_df = self.xml_to_pd_genes("../data/en_product6.xml")
        self.merge_data_frames()
    def download_orpha_file(self, name: str) -> None:
        if os.path.exists(f"../data/{name}"):
            print(f"{name} previously downloaded")
            return
        output_file = f"../data/{name}"
        orpha_url = f"https://www.orphadata.com/data/xml/{name}"
        response = requests.get(orpha_url)
        response.raise_for_status()  # will throw an error if download fails
        with open(output_file, "wb") as f:
            f.write(response.content)
        print(f"File saved as {output_file}")
    def xml_to_pd_prevalence(self, filename: str) -> pd.DataFrame:
        tree = ET.parse(filename)
        root = tree.getroot()
        records = []
        for disorder in root.findall(".//Disorder"):
            orpha_code = disorder.findtext("OrphaCode")
            name = disorder.find("Name").text # type: ignore
            for prev in disorder.findall(".//Prevalence"):
                records.append({
                    "OrphaCode": orpha_code,
                    "Name": name,
                    "Prevalence": prev.findtext("PrevalenceClass/Name"),
                    "Geo": prev.findtext("PrevalenceGeographic/Name")
                })
        df = pd.DataFrame(records)
        return df
    def xml_to_pd_onset(self, filename: str) -> pd.DataFrame:
        tree = ET.parse(filename)
        root = tree.getroot()
        records = []
        for disorder in root.findall(".//Disorder"):
            orpha_code = disorder.findtext("OrphaCode")
            name = disorder.find("Name").text # type: ignore
            # Collect AverageAgeOfOnset
            ages = [
                onset.findtext("Name")
                for onset in disorder.findall(".//AverageAgeOfOnset")
            ]
            # Collect TypeOfInheritance
            inheritances = [
                inh.findtext("Name")
                for inh in disorder.findall(".//TypeOfInheritance")
            ]
            # One record per combination (cartesian product)
            if ages and inheritances:
                for age in ages:
                    for inh in inheritances:
                        records.append({
                            "OrphaCode": orpha_code,
                            "Name": name,
                            "AverageAgeOfOnset": age,
                            "TypeOfInheritance": inh,
                        })
            elif ages:
                for age in ages:
                    records.append({
                        "OrphaCode": orpha_code,
                        "Name": name,
                        "AverageAgeOfOnset": age,
                        "TypeOfInheritance": None,
                    })
            elif inheritances:
                for inh in inheritances:
                    records.append({
                        "OrphaCode": orpha_code,
                        "Name": name,
                        "AverageAgeOfOnset": None,
                        "TypeOfInheritance": inh,
                    })
            else:
                records.append({
                    "OrphaCode": orpha_code,
                    "Name": name,
                    "AverageAgeOfOnset": None,
                    "TypeOfInheritance": None,
                })
        df = pd.DataFrame(records)
        return df
    def xml_to_pd_genes(self, filename) -> pd.DataFrame:
        tree = ET.parse(filename)
        root = tree.getroot()
        records = []
        for disorder in root.findall(".//Disorder"):
            orpha_code = disorder.findtext("OrphaCode")
            disorder_name = disorder.find("Name").text # type: ignore
            for assoc in disorder.findall(".//DisorderGeneAssociation"):
                gene = assoc.find("Gene")
                if gene is None:
                    continue
                gene_symbol = gene.findtext("Symbol")
                gene_name = gene.findtext("Name")
                # Collect synonyms
                synonyms = [syn.text for syn in gene.findall(".//Synonym")]
                # Collect locus (may be multiple)
                loci = [loc.findtext("GeneLocus") for loc in gene.findall(".//Locus")]
                # Collect external references (as tuples Source:Ref)
                ext_refs = [
                    f"{ref.findtext('Source')}:{ref.findtext('Reference')}"
                    for ref in gene.findall(".//ExternalReference")
                ]
                records.append({
                    "OrphaCode": orpha_code,
                    "DisorderName": disorder_name,
                    "GeneSymbol": gene_symbol,
                    "GeneName": gene_name,
                    "Synonyms": "; ".join(synonyms) if synonyms else None, # type: ignore
                    "Locus": "; ".join(loci) if loci else None, # type: ignore
                    "ExternalRefs": "; ".join(ext_refs) if ext_refs else None,
                })
        df = pd.DataFrame(records)
        return df
    def merge_data_frames(self) -> None:
        df = self._genes_df.merge(self._onset_df, on="OrphaCode", how="left").merge(self._prevalence_df, on="OrphaCode", how="left")
        df = df[["OrphaCode", "DisorderName",   "GeneSymbol", "GeneName", "Prevalence", "Geo", "AverageAgeOfOnset"]]
        df = df[df["Prevalence"].notna()]
        self._df = df
    def get_prev(self) -> pd.DataFrame:
        return self._prevalence_df
    def get_onsets(self) -> pd.DataFrame:
        return self._onset_df
    def get_genes(self) -> pd.DataFrame:
        return self._genes_df
    def get_merged(self) -> pd.DataFrame:
        return self._df
    def get_sum_of_prevalnces(self, cell_value) -> float:
        fields = cell_value.split(";")
        items = [i.strip() for i in fields]
        result = 0
        for item in items:
            if item not in prevalence_map:
                raise ValueError(f"Malformed prevalence map: '{item}'")
            else:
                result += prevalence_map[item]
        return result

In [29]:
# map prevelance to patients
orpha_prev = OrphaPrev()

print(orpha_prev.get_merged().head())
# get dict of orpha code to mapped prevelance
orpha_code_to_prevalence_dirty = orpha_prev.get_merged().set_index("OrphaCode")["Prevalence"].to_dict()
orpha_code_to_prevalence = {}
for orpha_code, prev in orpha_code_to_prevalence_dirty.items():
    orpha_code = f"ORPHA:{orpha_code}"
    if prev in prevalence_category:
        prev = prevalence_category[prev]
        orpha_code_to_prevalence[orpha_code] = prev

print(list(orpha_code_to_prevalence.items())[0:5])

en_product9_prev.xml previously downloaded
en_product9_ages.xml previously downloaded
en_product6.xml previously downloaded
  OrphaCode                                       DisorderName GeneSymbol  \
1    166024  Multiple epiphyseal dysplasia-macrocephaly-fac...       KIF7   
3    166024  Multiple epiphyseal dysplasia-macrocephaly-fac...       KIF7   
4        93                             Aspartylglucosaminuria        AGA   
5        93                             Aspartylglucosaminuria        AGA   
6        93                             Aspartylglucosaminuria        AGA   

                  GeneName       Prevalence        Geo AverageAgeOfOnset  
1  kinesin family member 7   <1 / 1 000 000  Worldwide           Infancy  
3  kinesin family member 7   <1 / 1 000 000  Worldwide          Neonatal  
4  aspartylglucosaminidase  1-9 / 1 000 000  Australia         Childhood  
5  aspartylglucosaminidase          Unknown  Worldwide         Childhood  
6  aspartylglucosaminidase  1-9 / 1 00

In [46]:
rare = []
common = []
unmapped = 0
for patient_id, orpha_id in list(patient_to_orpha.items()):
    if orpha_id in orpha_code_to_prevalence:
        prevalence = orpha_code_to_prevalence[orpha_id]
        if prevalence == "RARE":
            rare.append(patient_filename[patient_id])
        elif prevalence == "COMMON":
            common.append(patient_filename[patient_id])
    else:
        unmapped += 1

print(f"Total patients: {len(patient_to_orpha)}, Rare: {len(rare)}, Common: {len(common)}, Unmapped: {unmapped}")
# we need to check the phenopacket

# write the two list of patitent ids
# create output directory if it does not exist
os.makedirs(phenopacket_out, exist_ok=True)
with open(os.path.join(phenopacket_out, "phenopackets_rare.txt"), "w") as f:
    for pid in rare:
        f.write(f"{pid}\n")
with open(os.path.join(phenopacket_out, "phenopackets_common.txt"), "w") as f:
    for pid in common:
        f.write(f"{pid}\n")

Total patients: 3185, Rare: 1532, Common: 40, Unmapped: 1613
